In [2]:
from mlflow_utils import setup, test_data, team_train_test_split, atlantic_division
from mlflow_utils.metrics import classification, regression
import mlflow
# import model
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import pandas as pd
import numpy as np
import warnings
from sklearn.metrics import accuracy_score

warnings.filterwarnings("ignore")

setup() # leave blank, if doing covid or win prob then pass 'covid' or 'win'

In [3]:
model_name = "randomforest" # place name of model here eg xgboost
problem_type = "classification" # place one of [regression, classification, ts] here

In [5]:
data = test_data(data=problem_type) # returns real dataset

In [6]:
# split data
team = ''       # only needed if doing regression
if problem_type == 'regression':
    opponents = [t for t in atlantic_division if t != team]
    x_train, x_test, y_train, y_test = team_train_test_split(team=team, opponents=opponents, data=data)
else:
    x_train, x_test, y_train, y_test = train_test_split(data.data.drop([data.target], axis=1), data.data[data.target], test_size=.25)

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True, False],
    'max_features': ['auto', 2, 3, 5, 6, 9],
    'min_samples_leaf': [1, 3, 4, 5, 6, 7],
    'min_samples_split': [2, 8, 10, 12, 14, 16],
    'n_estimators': [100, 200, 300, 1000],
    'max_depth' : [0, 2, 4, 6, 8, 12]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
ran_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
ran_search.fit(x_train,y_train)

print(ran_search.best_params_)
print(ran_search.best_score_)

Fitting 3 folds for each of 10368 candidates, totalling 31104 fits


In [46]:
model = RandomForestClassifier(bootstrap= True, max_features= 2, min_samples_leaf= 1, min_samples_split= 12, n_estimators= 100)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print(accuracy_score(y_test, y_pred))

0.6098484848484849


In [48]:
if accuracy_score(y_test, y_pred) > best_score:
    best_params = ran_search.best_params_
    best_score = accuracy_score(y_test, y_pred)
    print('Beaten')
print(best_params)
print(best_score)

{'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 3, 'bootstrap': True}
0.6098484848484849


In [13]:
mlflow.autolog(silent=True)
# initialize model



with mlflow.start_run() as run:
    mlflow.tracking.MlflowClient().set_tag(run_id=run.info.run_id, key=model_name, value=problem_type)
    # train model
    classi.fit(x_train, y_train)
    y_pred = classi.predict(x_test)
    

    # run classification/regression metrics to log results
    metrics = classification(y_test, y_pred)


In [14]:
print(metrics)

{'accuracy': 0.6212121212121212, 'precision': 0.66875, 'auc': 0.6064935064935064, 'f1': 0.6815286624203821, 'recall': 0.6948051948051948}


In [ ]:
#https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
#https://blog.dataiku.com/narrowing-the-search-which-hyperparameters-really-matter
#https://rspiro9.github.io/hyperparameter_tuning_for_random_forest